In [ ]:
from src.pano_llava_processor import PanoLLaVAProcessor
from src.dataset import ChatPanoDataset
from src.processor.image import PanoramaImageProcessor
from src.processor.text import TextTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator

# 1) 공용 Processor 준비
img_proc = PanoramaImageProcessor(image_size=(224, 224), crop_strategy="e2p")
txt_tok  = TextTokenizer("Qwen/Qwen3-0.6B")
pano_proc= PanoLLaVAProcessor(img_proc, txt_tok)          # vis_proc 생략

# 2) Dataset 인스턴스
ds = ChatPanoDataset(
    csv_path   = "data/quic360/downtest.csv",          # url,query,annotation 형식
    
    processor  = pano_proc,
    tokenizer  = txt_tok.tok,          # 프롬프트 빌드에 사용
    system_msg = "You are a helper.",  # ⤷ system 메시지 변경 가능
    flatten    = True                  # pixel_values → (B·V, C, H, W)
)

# 3) PyTorch DataLoader
dl = DataLoader(ds, batch_size=2, shuffle=True,
                collate_fn=default_data_collator, num_workers=4)

batch = next(iter(dl))

/Users/seungwoo/anaconda3/envs/sur/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(batch.keys())
# 
print("pixel_values shape:", batch["pixel_values"].shape)  # (B·V, C, H, W)
# print("pixel_values_5d shape:", batch["pixel_values_5d"].shape)  # (B, V, C, H, W) --- IGNORE ---
print("input_ids shape:", batch["input_ids"].shape)  # (B, L)
print("attention_mask shape:", batch["attention_mask"].shape)  # (B, L)
print("labels shape:", batch["labels"].shape)  # (B, L)
# dict_keys(['pixel_values', 'input_ids', 'attention_mask'])

dict_keys(['pixel_values', 'input_ids', 'attention_mask', 'labels'])
pixel_values shape: torch.Size([2, 8, 3, 224, 224])
input_ids shape: torch.Size([2, 1, 131072])
attention_mask shape: torch.Size([2, 1, 131072])
labels shape: torch.Size([2, 1, 131072])
